In [1]:
import pandas as pd

In [2]:
trainData = pd.read_csv('train.csv')

In [3]:
trainData.head()

,Comment,Class
0,"""You fuck your dad.""",1
1,"""i really don't understand your point.\xa0 It ...",0
2,"""A\\xc2\\xa0majority of Canadians can and has ...",0
3,"""listen if you dont wanna get married to a man...",0
4,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd...",0


In [31]:
from bert_serving.client import BertClient
bc = BertClient()
bc.encode(['First do it', 'then do it right', 'then do it better'])

array([[-1.4607856 , -1.3356979 ,  0.15181774, ...,  0.7314374 ,
         0.95083904, -0.22632086],
       [-1.2321306 , -1.3150991 ,  0.10333592, ...,  0.5036387 ,
         1.1127867 ,  0.01645399],
       [-1.3529371 , -1.5694213 ,  0.04423429, ...,  0.39017606,
         1.0482806 ,  0.14896075]], dtype=float32)

In [5]:
trainData['BertEncoded'] = trainData.Comment.apply(lambda x: bc.encode([x])[0])

newDf = pd.DataFrame(list(trainData.BertEncoded), columns = ["Col" + str(i) for i in range(0,768)])
newDf["Insult"] = trainData.Class
newDf.to_csv('BertEncodedData.csv', index=False)

/home/xavient/anaconda3/lib/python3.7/site-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [8]:
newDf.head()

,Col0,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,...,Col759,Col760,Col761,Col762,Col763,Col764,Col765,Col766,Col767,Insult
0,-1.613887,-0.954983,-0.033115,-0.482596,-0.252641,-0.506401,-0.436636,1.444905,0.640566,-0.702963,...,1.240674,0.642965,-0.221781,0.872750,0.864134,-1.230446,0.020024,1.465738,-0.029244,1
1,-1.447952,-0.920585,0.423175,-0.874580,-0.319325,-0.443531,-0.445665,1.106715,0.274117,-0.598053,...,1.341768,0.552965,-0.638997,0.704303,1.099262,-0.795739,0.057477,0.996351,-0.038786,0
2,-1.557941,-0.690302,0.395983,-0.933049,-0.189672,-0.308341,-0.561640,1.038212,0.060139,-0.981285,...,1.515023,0.507542,-0.727956,0.642189,0.965338,-0.946272,0.264077,1.226565,0.103484,0
3,-1.561248,-0.978212,0.344685,-0.904927,-0.382290,-0.481175,-0.559459,1.256119,0.355284,-0.525047,...,1.653658,0.504713,-0.556801,0.792939,0.978399,-0.740440,0.202388,1.039600,0.122538,0
4,-1.296834,-0.838971,-0.016590,-0.774820,-0.389915,-0.393829,-0.518376,1.075417,0.443116,-1.078864,...,1.749387,0.675712,-0.547855,0.608583,1.078333,-0.994598,0.195915,1.256632,0.176020,0


In [10]:
newDf.shape

(11476, 769)

In [9]:
from sklearn.linear_model import LogisticRegression

In [11]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test , Y_train, Y_test = train_test_split(newDf.drop(['Insult'], axis = 1), newDf['Insult'], 
                                                     test_size=0.2, random_state=42)

In [16]:
X_train.shape, Y_train.shape , X_test.shape, Y_test.shape

((9180, 768), (9180,), (2296, 768), (2296,))

In [14]:
lr = LogisticRegression()

In [17]:
lr.fit(X_train, Y_train)

/home/xavient/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
lr.score(X_test,Y_test)

0.7931184668989547

In [19]:
from sklearn.metrics import classification_report

In [21]:
print(classification_report(Y_test, lr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.82      0.91      0.86      1586
           1       0.72      0.54      0.62       710

    accuracy                           0.79      2296
   macro avg       0.77      0.72      0.74      2296
weighted avg       0.79      0.79      0.78      2296



In [27]:
texts = [
        ]

In [28]:
valData = bc.encode(texts)

In [30]:
for i in zip(texts, lr.predict(valData)):
    print(i)

('This is fucking awesome', 0)
('Fuck you', 1)
('Shut up', 1)
('Shut the door', 0)
('F**k yeah', 0)
('F*ck yeah', 0)
('F**k you', 1)
